<a href="https://colab.research.google.com/github/shirin-menon/assign/blob/main/skipgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import decomposition
from pathlib import Path
import seaborn as sns
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)
import numpy as np
import pandas as pd

corpus=[ 'drink water',
        'drink juice',
        'drink milkshake',
        'eat snacks',
        'eat food',
        'eat rice',
        'help old people',
        'help kids',
        'help everyone',
        'she is a girl',
        'he is a man',
        'im a king',
        'she is a guard',
        'i like mango',
        'i could use some help',
        'i would like some juice',
        'lotus is the national flower of india',
        'tiger is the national animal of india',
        'nepal is neighboring country of india'
        ]
def vocab_words(corpus):
  vocabulary={}
  i=0
  for s in corpus:
     for w in s.split():
       if w not in vocabulary:
         vocabulary[w]=i
         i+=1
  return vocabulary

def create_set(corpus, n_gram=1):
  columns=['Input'] + [f'Output{i+1}' for i in range(n_gram*2)]
  res=pd.DataFrame(columns=columns)
  for sentence in corpus:
    for i,w in enumerate(sentence.split()):
      inp=[w]
      out=[]
      for n in range(1, n_gram+1):
        if (i-1)>=0:
          out.append(sentence.split()[i-n])
        else:
          out.append('<padding>')
        if (i+n)<len(sentence.split()):
           out.append(sentence.split()[i+n])
        else:
           out.append('<padding>')
    row = pd.DataFrame([inp+out], columns = columns)
    res = res.append(row, ignore_index = True)
  return res


def prep_set_ravel(corpus, n_gram=1):
   columns=['Input','Output']
   res=pd.DataFrame(columns=columns)
   for sentence in corpus:
     for i,w in enumerate(sentence.split()):
       inp=w
       for n in range(1, n_gram+1):
         if (i-n)>=0:
           out=sentence.split()[i-n]
           row = pd.DataFrame([[inp,out]], columns = columns)
           res = res.append(row, ignore_index = True)

         if (i+n)<len(sentence.split()):
            out = sentence.split()[i+n]
            row = pd.DataFrame([[inp,out]], columns = columns)
            res = res.append(row, ignore_index = True)
     return res

stop_words=set(stopwords.words('english'))
def preprocess(corpus):
    result = []
    for i in corpus:
        out = nltk.word_tokenize(i)
        out = [x.lower() for x in out]
        out = [x for x in out if x not in stop_words]
        result.append(" ". join(out))
    return result


corpus = preprocess(corpus)
corpus
vocabulary = vocab_words(corpus)
vocabulary
train_emb = create_set(corpus, n_gram = 2)
train_emb.head()
train_emb = prep_set_ravel(corpus, n_gram = 2)
train_emb.head()
train_emb.Input = train_emb.Input.map(vocabulary)
train_emb.Output = train_emb.Output.map(vocabulary)
train_emb.head()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Input,Output
0,0,1
1,1,0
